# ArrayFire Example : BlackScholes

Now let us work on a simple example, to show you what ArrayFire can do. 

### BlackScholes

This is the powerhouse of modern financial simulations. From this model, we can estimate the theoretical price of a European style option. This model is fairly used (with some modifications and tuning) by options markets participants. 

Now we're going to write a simple blacksholes kernel, and accelerate it using ArrayFire.

In [1]:
using ArrayFire

In [2]:
T = Float32

Float32

Write a simple blackscholes kernel.

In [3]:
function blackscholes_serial(sptprice::T,
                            strike::AbstractArray,
                            rate::T,
                            volatility::T,
                            time::T)
    logterm = log10( sptprice ./ strike)
    powterm = .5f0 .* volatility .* volatility
    den = volatility .* sqrt(time)
    d1 = (((rate .+ powterm) .* time) .+ logterm) ./ den
    d2 = d1 .- den
    NofXd1 = cndf2(d1)
    NofXd2 = cndf2(d2)
    futureValue = strike .* exp(- rate .* time)
    c1 = futureValue .* NofXd2
    call_ = sptprice .* NofXd1 .- c1
    put  = call_ .- futureValue .+ sptprice
end

@inline function cndf2(in::AbstractArray)
    out =  1/2 + erf(1/√2 .* in) ./ 2
    return out
end

cndf2 (generic function with 1 method)

Create all the arrays: spot price, various initial strike prices, rates and volatility, and time period. We have chosen a million iterations to simulate. 

In [4]:
iterations = 10^7
sptprice   = T(42.0)
initStrike = T[ 40.0 + (i / iterations) for i = 1:iterations ]
rate       = T(0.5) 
volatility = T(0.2) 
time       = T(0.5) 

0.5f0

Let's now convert them all to GPU arrays. Now, all these different parameters now reside on the GPU. 

In [5]:
initStriked = AFArray(initStrike)

10000000-element ArrayFire.AFArray{Float32,1}:
 40.0
 40.0
 40.0
 40.0
 40.0
 40.0
 40.0
 40.0
 40.0
 40.0
 40.0
 40.0
 40.0
  ⋮  
 41.0
 41.0
 41.0
 41.0
 41.0
 41.0
 41.0
 41.0
 41.0
 41.0
 41.0
 41.0

Let's now run the black scholes simulation, and time the execution.

In [6]:
@time begin 
    put1 = blackscholes_serial(sptprice, initStrike, rate, volatility, time)
    @show sum(put1)
end

sum(put1) = 2.0954822f8
  1.848766 seconds (360.94 k allocations: 741.225 MB, 6.26% gc time)


2.0954822f8

Now let's run the accelerated blackscholes simulation. Notice that we don't need to change any code. All we need to do is feed in different inputs. 

This is **multiple dispatch** at work.

In [7]:
@time begin 
    put2 = blackscholes_serial(sptprice, initStriked, rate, volatility, time)
    @show sum(put2)
end

sum(put2) = 2.0954820959255382e8
  0.581040 seconds (418.75 k allocations: 18.451 MB)


2.0954820959255382e8